In [1]:
# !wget https://object.pouta.csc.fi/OPUS-GNOME/v1/raw/en.zip
# !wget https://object.pouta.csc.fi/OPUS-GNOME/v1/raw/ms.zip
# !unzip -o en.zip
# !unzip -o ms.zip

In [2]:
# !wget https://object.pouta.csc.fi/OPUS-GNOME/v1/xml/en-ms.xml.gz
# !gzip -d en-ms.xml.gz

In [3]:
from bs4 import BeautifulSoup
from tqdm import tqdm

with open('en-ms.xml') as fopen:
    xml = fopen.read().split('\n')
    
mapping, ids = {}, []
enable = False
for row in tqdm(xml):
    if '<linkGrp targType' in row:
        s = BeautifulSoup(row)
        r = 'GNOME/raw/' + s.findAll('linkgrp')[0].get('fromdoc').replace('.gz','')
        l = 'GNOME/raw/' + s.findAll('linkgrp')[0].get('todoc').replace('.gz','')
        enable = True
    if '<link id' in row and enable:
        s = BeautifulSoup(row)
        t = s.findAll('link')[0].get('xtargets')
        t = list(filter(None, t.split(';')))
        if len(t) != 2:
            continue
        ids.append([t[0].split(), t[1].split()])
    if '</linkGrp>' in row:
        label = f'{l} <> {r}'
        mapping[label] = ids
        enable = False
        ids = []

100%|██████████| 306166/306166 [01:11<00:00, 4267.58it/s]


In [4]:
mapping[label]

[[['s4'], ['s4']],
 [['s5'], ['s5']],
 [['s6'], ['s6']],
 [['s12'], ['s12']],
 [['s19'], ['s19']],
 [['s22'], ['s22']],
 [['s23'], ['s23']],
 [['s24'], ['s24']],
 [['s25'], ['s25']],
 [['s26'], ['s26']],
 [['s27'], ['s27']],
 [['s28'], ['s28']],
 [['s29'], ['s29']],
 [['s31'], ['s31']],
 [['s32'], ['s32']],
 [['s33'], ['s33']],
 [['s34'], ['s34']],
 [['s35'], ['s35']],
 [['s36'], ['s36']],
 [['s37'], ['s37']],
 [['s38'], ['s38']],
 [['s39'], ['s39']],
 [['s40'], ['s40']],
 [['s41'], ['s41']],
 [['s42'], ['s42']],
 [['s43'], ['s43']],
 [['s44'], ['s44']],
 [['s45'], ['s45']],
 [['s46'], ['s46']],
 [['s47'], ['s47']],
 [['s48'], ['s48']],
 [['s50'], ['s50']],
 [['s53'], ['s53']],
 [['s54'], ['s54']],
 [['s56'], ['s56']],
 [['s57'], ['s57']],
 [['s58'], ['s58']],
 [['s60'], ['s60']],
 [['s62'], ['s62']],
 [['s64'], ['s64']],
 [['s66'], ['s66']],
 [['s67'], ['s67']],
 [['s68'], ['s68']],
 [['s69'], ['s69']],
 [['s70'], ['s70']],
 [['s75'], ['s75']],
 [['s76'], ['s76']],
 [['s77'], ['s77']]

In [5]:
results = []
for file in tqdm(mapping.keys()):
    l, r = file.split(' <> ')
    ids = mapping[file]
    try:
        with open(l) as fopen:
            left = fopen.read()
        soup_left = BeautifulSoup(left, 'html.parser')
        with open(r) as fopen:
            right = fopen.read()
        soup_right = BeautifulSoup(right, 'html.parser')
        left_dict = {}
        for s in soup_left.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            left_dict[i] = text
        right_dict = {}
        for s in soup_right.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            right_dict[i] = text
        combined = []
        for i in ids:
            try:
                r = ' '.join([right_dict[k] for k in i[0]])
                l = ' '.join([left_dict[k] for k in i[1]])
                combined.append((l, r))
            except:
                pass
        results.extend(combined)
    except Exception as e:
        print(e)
        pass

100%|██████████| 898/898 [01:08<00:00, 13.09it/s]


In [6]:
len(results)

302567

In [7]:
results[:20]

[('GNOME', 'GNOME'),
 ('Tema GNOME Default', 'Default GNOME Theme'),
 ('OK', 'OK'),
 ('Seni', 'Art'),
 ('Kamera', 'Camera'),
 ('Diperakui', 'Certified'),
 ('Sejuk', 'Cool'),
 ('Bahaya', 'Danger'),
 ('Desktop', 'Desktop'),
 ('Unggul', 'Distinguished'),
 ('Draf', 'Draft'),
 ('Jabat tangan', 'Handshake'),
 ('Pemasaran', 'Marketing'),
 ('Wang', 'Money'),
 ('Multimedia', 'Multimedia'),
 ('Nota', 'Note'),
 ('Alamak!', 'Oh no!'),
 ('Orang', 'People'),
 ('Peribadi', 'Personal'),
 ('Gambar', 'Pictures')]

In [8]:
import json

with open('gnome-ms-en.json', 'w') as fopen:
    json.dump(results, fopen)

In [9]:
!rm -rf GNOME
!rm -rf ms.zip en.zip en-ms.xml.gz en-ms.xml